# **Colabの初期設定**
`Kaggle`データセットのダウンロードはここから`GoogleDrive`上に行う。

ダウンロード後のデータ分析は、各ノートブック`.ipynb`を作成して行う。

その際、`GoogleDrive`のマウントは各ノート上で実行する。

下記は、`KaggleAPI`の有効化、`GoogleDrive`のマウント化、`Tree`モジュールの追加、`Kaggle`データセットのダウンロードの実施手順。

## **`Kaggle API`のインストール**

In [12]:
!pip install kaggle

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


## **`Kaggle API`の有効化**

[Kaggle API with Colab](https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=5l1V_oxXsZ8l&forceEdit=true&sandboxMode=true)

下記実行前に、`kaggle.json`をあらかじめDLし、`GoogleDrive`に格納しておく。実行すると認証設定が呼び出され、許可すると`GoogleDrive`ディレクトリ内から`kaggle.json`ファイルが検索され、`root/.kaggle`以下に格納される。
元々のコードだと

`filename = "/content/.kaggle/kaggle.json"`

となっているが、API起動時に参照エラーが発生するため

`filename = "/root/.kaggle/kaggle.json"`

へ変更する事。


In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


## **`GoogleDrive`のマウント**
先に`GoogleDrive`を`Colab上へ`マウントした場合、`Googledrive`上の`kaggle.json`内の記載が空白化する事象が発生したため、`KaggleAPI`導入後に実施

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## **`Tree`パッケージのインストール**

ディレクトリ構成の記載に便利なため導入

In [0]:
!apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (60.6 kB/s)
Selecting previously unselected package tree.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


## ディレクトリ構成

`My Drive`以下は、`GoogleDrive`のマウント先。データを保存することで、
`GoogleDrive`に同期される。

`Colab Notebooks`：Notebookの格納

`datasets`：各データセットの格納

`setting`：設定ファイルの格納

In [0]:
!tree -d

.
├── drive
│   └── My Drive
│       ├── data_analysis-colab
│       │   ├── Colab Notebooks
│       │   ├── kaggle
│       │   │   └── titanic
│       │   ├── ml100knock
│       │   │   ├── 10_Questionnaire_analysis
│       │   │   ├── 1_web_order
│       │   │   ├── 2_Retail_data
│       │   │   ├── 3_Customer_information
│       │   │   ├── 4_ Customer_behavior
│       │   │   ├── 5_Customer_withdrawal
│       │   │   ├── 6_Logistics_route
│       │   │   ├── 7_ Logistics_network
│       │   │   ├── 8_ Numerical_simulation
│       │   │   └── 9_Potential_customer
│       │   │       ├── img
│       │   │       └── mov
│       │   └── setting
│       ├── github
│       ├── private
│       └── remote-colab
│           ├── Colab Notebooks
│           ├── datasets
│           │   ├── kaggle
│           │   │   └── titanic
│           │   └── ml100knock
│           │       ├── 10_Questionnaire_analysis
│           │       ├── 1_web_order
│           │       ├── 2_Retail_data
│           

## **`Kaggle`データセットのダウンロード**

使用するデータセットは`./drive/My\ Drive/datasets/kaggle/`以下に格納

`!kaggle competitions download -h`

```
usage: kaggle competitions download [-h] [-f FILE_NAME] [-p PATH] [-w] [-o]
                                    [-q]
                                    [competition]

optional arguments:
  -h, --help            show this help message and exit
  competition           Competition URL suffix (use "kaggle competitions list" to show options)
                        If empty, the default competition will be used (use "kaggle config set competition")"
  -f FILE_NAME, --file FILE_NAME
                        File name, all files downloaded if not provided
                        (use "kaggle competitions files -c <competition>" to show options)
  -p PATH, --path PATH  Folder where file(s) will be downloaded, defaults to current working directory
  -w, --wp              Download files to current working path
  -o, --force           Skip check whether local version of file is up to date, force file download
  -q, --quiet           Suppress printing information about the upload/download progress
```






In [0]:
!kaggle competitions download -c titanic -p ./drive/My\ Drive/githyb/datasets/kaggle/titanic/

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 438kB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 8.30MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 6.26MB/s]


In [32]:
# クローンgithubのリポジトリのセットアップ
# インポートはROOTパスとMY_GOOGLE_DRIVE_PATHを結合するために使用さジョイン
from os.path import join

ROOT = '/content/drive'
# Googleドライブ上のプロジェクトへのパス
MY_GOOGLE_DRIVE_PATH = 'My Drive/remote-colab/ ' 
# あなたのGithubのユーザー名と交換
GIT_USERNAME = "otompton" 
# 間違いなくあなたの
GIT_TOKEN = "669c6c60d72e3214cff5cbf0b7d326763346c308"  

# この場合、githubリポジトリに置き換えます
#  deep-learning-v2-pytorchリポジトリを複製する
GIT_REPOSITORY = "remote-colab" 

PROJECT_PATH = join(ROOT,MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)


PROJECT_PATH:  /content/drive/My Drive/remote-colab/ 
mkdir: cannot create directory ‘/content/drive/My Drive/remote-colab/ ’: File exists
GIT_PATH:  https://669c6c60d72e3214cff5cbf0b7d326763346c308@github.com/otompton/remote-colab.git


In [0]:
!mkdir /content/drive/My\ Drive/remote-colab

In [38]:
!git clone "{GIT_PATH}"

Cloning into 'remote-colab'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [39]:
cd remote-colab/

/content/drive/My Drive/remote-colab


In [40]:
ls -a

.git/  README.md


In [0]:
cd remote-colab/